In [2]:
#evansdata
#gather time stamps --camera and wf--
import numpy as np
import os
import pandas as pd
import h5py
from datapipe import find_nearest, getData, rowsToColumns
from pathlib import Path
import pickle
import scipy.io as scio
from sklearn.externals import joblib
from skimage import io
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"]=""; 
import tensorflow as tf
%load_ext autoreload
%autoreload 2

In [3]:
import os
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential, load_model, Model
from keras.layers import Input, Dense, GRU, Embedding, LSTM, Dropout
from keras.optimizers import RMSprop
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau

Using TensorFlow backend.


In [4]:
#should respond to audio

im1 = io.imread('./data/whisk_AV2_darker.tif')
(length, y, x) = im1.shape

In [5]:
print(length, y, x)

53232 123 130


In [6]:
im2 = io.imread('./data/eye_AV2.tif')

In [ ]:
im3 =io.imread('./data/ear_AV2.tif')

In [7]:
videoTS = np.ndarray.flatten(scio.loadmat('./data/camera_ts.mat')['cameraTime'])
wf_ts = np.ndarray.flatten(scio.loadmat('./data/wf_ts.mat')['cropped_wf'])

In [8]:
videoAns = []
for vidInd, vidTime in enumerate(wf_ts):
    windowT = vidTime
    if(vidTime > videoTS[0]) and (vidTime+3.85< videoTS[-1]):
        currIdx = find_nearest(videoTS, windowT)
        videoAns.append(currIdx)

In [9]:
vidIndexNP = im1[videoAns]
np.save("./data/whisk_df2.npy", vidIndexNP)

In [10]:
#either load above data set or if it has jus tbeen run then dont
#here
eyeIndexNP = im2[videoAns]
np.save("./data/eye_df2.npy", eyeIndexNP)

In [ ]:
earIndexNP = im3[videoAns]
np.save("./data/ear_df.npy", earIndexNP)

In [11]:
# whiskEncoder
whiskEncoder = load_model('./data/whiskEncoder2.h5')
input_img = Input(shape=(126, 132, 1)) 
w_l  = whiskEncoder.layers[0]
w_l1 = whiskEncoder.layers[1]
w_l2 = whiskEncoder.layers[2]
w_l3 = whiskEncoder.layers[3]
w_l4 = whiskEncoder.layers[4]
w_l5 = whiskEncoder.layers[5]

whisk_encoder = Model(input_img, w_l5(w_l4(w_l3(w_l2(w_l1(w_l(input_img)))))))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [16]:
earEncoder = load_model('./data/earEncoder2.h5')


OSError: Unable to open file (unable to open file: name = './data/earEncoder2.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [24]:
eyeEncoder = load_model('./data/eyeEncoder2.h5')
input_img = Input(shape=(64, 84, 1)) 
e_l  = eyeEncoder.layers[0]
e_l1 = eyeEncoder.layers[1]
e_l2 = eyeEncoder.layers[2]
e_l3 = eyeEncoder.layers[3]
e_l4 = eyeEncoder.layers[4]
e_l5 = eyeEncoder.layers[5]
eye_encoder = Model(input_img, e_l5(e_l4(e_l3(e_l2(e_l1(e_l(input_img)))))))

In [20]:
def cropImage(imArr):
    (length, x, y, channel) = imArr.shape
    cropped = np.zeros((length, x+2, y, channel))
    for ind, im in enumerate(imArr):
        for rowind, row in enumerate(im):
            cropped[ind][rowind] = row.reshape(y,1)
    return cropped

In [13]:
#get the size of the output of the last layer, create an array of that size 
#then for each image get the encoding
vidIndexNP = vidIndexNP.reshape(len(videoAns), 123,130,1)
vidIndexNPcrop = cropImage(vidIndexNP)
encoded_images = whisk_encoder.predict(vidIndexNPcrop)
np.save("./data/whisk_encoded_df2.npy", encoded_images)

In [23]:
eyeIndexNP = eyeIndexNP.reshape(len(videoAns), 62, 84,1)
eyeIndexNPcrop = cropImage(eyeIndexNP)
eye_encoded = eye_encoder.predict(eyeIndexNPcrop)
np.save("./data/eye_encoded_df2.npy", eye_encoded)

In [ ]:
earIndexNP = earIndexNP.reshape(len(videoAns), , ,1)
ear_encoded = ear_encoder2.predict(earIndexNP)
np.save("./data/eye_encoded_df2.npy")